In [18]:
import pandas as pd
import numpy as np
import json

seasons = range(2021, 2023)
zones = range(1, 31)

gplus_data = pd.DataFrame()

In [19]:
for yr in seasons:
    print(f"Grabbing {yr}")
    for z in zones:
        url = f"https://app.americansocceranalysis.com/api/v1/mls/teams/goals-added?zone={z}&season_name={yr}&stage_name=Regular%20Season"
        print(f"{yr} - accessing ASA zone: {z}")
        tmp = pd.read_json(url)
        tmp['zone'] = z
        tmp['season_name'] = yr
        gplus_data = gplus_data.append(tmp, ignore_index=True)

gplus_data

Grabbing 2021
2021 - accessing ASA zone: 1
2021 - accessing ASA zone: 2
2021 - accessing ASA zone: 3
2021 - accessing ASA zone: 4
2021 - accessing ASA zone: 5
2021 - accessing ASA zone: 6
2021 - accessing ASA zone: 7
2021 - accessing ASA zone: 8
2021 - accessing ASA zone: 9
2021 - accessing ASA zone: 10
2021 - accessing ASA zone: 11
2021 - accessing ASA zone: 12
2021 - accessing ASA zone: 13
2021 - accessing ASA zone: 14
2021 - accessing ASA zone: 15
2021 - accessing ASA zone: 16
2021 - accessing ASA zone: 17
2021 - accessing ASA zone: 18
2021 - accessing ASA zone: 19
2021 - accessing ASA zone: 20
2021 - accessing ASA zone: 21
2021 - accessing ASA zone: 22
2021 - accessing ASA zone: 23
2021 - accessing ASA zone: 24
2021 - accessing ASA zone: 25
2021 - accessing ASA zone: 26
2021 - accessing ASA zone: 27
2021 - accessing ASA zone: 28
2021 - accessing ASA zone: 29
2021 - accessing ASA zone: 30
Grabbing 2022
2022 - accessing ASA zone: 1
2022 - accessing ASA zone: 2
2022 - accessing ASA zo

,team_id,minutes,data,zone,season_name
0,0KPqjA456v,3395,"[{'action_type': 'Dribbling', 'num_actions_for...",1,2021
1,19vQ2095K6,3362,"[{'action_type': 'Dribbling', 'num_actions_for...",1,2021
2,9z5k7Yg5A3,3393,"[{'action_type': 'Dribbling', 'num_actions_for...",1,2021
3,a2lqR4JMr0,3378,"[{'action_type': 'Dribbling', 'num_actions_for...",1,2021
4,a2lqRX2Mr0,3402,"[{'action_type': 'Dribbling', 'num_actions_for...",1,2021
...,...,...,...,...,...
1645,WBLMvYAQxe,2503,"[{'action_type': 'Dribbling', 'num_actions_for...",30,2022
1646,X0Oq66zq6D,2374,"[{'action_type': 'Dribbling', 'num_actions_for...",30,2022
1647,YgOMngl5wN,2393,"[{'action_type': 'Dribbling', 'num_actions_for...",30,2022
1648,Z2vQ1xlqrA,2473,"[{'action_type': 'Dribbling', 'num_actions_for...",30,2022


In [23]:
json_expl_txt = json.loads(gplus_data.explode('data').to_json(orient="records"))
gplus_json_expl_flat = pd.json_normalize(json_expl_txt)
gplus_json_expl_flat.to_csv('./team-g+-zones.csv', index=False)
gplus_json_expl_flat.head()

,team_id,minutes,zone,season_name,data.action_type,data.num_actions_for,data.goals_added_for,data.num_actions_against,data.goals_added_against
0,0KPqjA456v,3395,1,2021,Dribbling,59,-0.0779,42,0.0076
1,0KPqjA456v,3395,1,2021,Fouling,9,0.0230,6,0.0148
2,0KPqjA456v,3395,1,2021,Interrupting,151,0.0703,124,0.2788
3,0KPqjA456v,3395,1,2021,Passing,201,0.3010,140,0.2040
4,0KPqjA456v,3395,1,2021,Receiving,131,0.0006,63,-0.0032


In [26]:
gplus_expl_flat = gplus_json_expl_flat[["season_name", "team_id", "minutes", "zone", "data.action_type", "data.goals_added_for", "data.goals_added_against"]]
gplus_expl_flat.columns = ["season_name",'team_id', 'minutes', 'zone', 'action_type', 'for_total', 'against_total']
gplus_expl_flat['for_p96'] = gplus_expl_flat["for_total"] * 96 / gplus_expl_flat["minutes"]
gplus_expl_flat['against_p96'] = gplus_expl_flat["against_total"] * 96 / gplus_expl_flat["minutes"]
gplus_expl_flat

/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,season_name,team_id,minutes,zone,action_type,for_total,against_total,for_p96,against_p96
0,2021,0KPqjA456v,3395,1,Dribbling,-0.0779,0.0076,-0.002203,0.000215
1,2021,0KPqjA456v,3395,1,Fouling,0.0230,0.0148,0.000650,0.000418
2,2021,0KPqjA456v,3395,1,Interrupting,0.0703,0.2788,0.001988,0.007884
3,2021,0KPqjA456v,3395,1,Passing,0.3010,0.2040,0.008511,0.005768
4,2021,0KPqjA456v,3395,1,Receiving,0.0006,-0.0032,0.000017,-0.000090
...,...,...,...,...,...,...,...,...,...
9020,2022,zeQZkL1MKw,2401,30,Fouling,0.1335,0.0210,0.005338,0.000840
9021,2022,zeQZkL1MKw,2401,30,Interrupting,0.0230,0.0515,0.000920,0.002059
9022,2022,zeQZkL1MKw,2401,30,Passing,-0.5158,-0.2697,-0.020623,-0.010784
9023,2022,zeQZkL1MKw,2401,30,Receiving,0.2175,0.1453,0.008696,0.005810


In [31]:
grouped_gplus = gplus_expl_flat.groupby(['season_name','team_id','zone']).agg({
    'minutes' : ['mean'],
    'for_total': ['sum'], 
    'against_total': ['sum'], 
    'for_p96': ['sum'], 
    'against_p96': ['sum']
}).reset_index()
grouped_gplus.columns = grouped_gplus.columns.droplevel(level=1)
grouped_gplus

,season_name,team_id,zone,minutes,for_total,against_total,for_p96,against_p96
0,2021,0KPqjA456v,1,3395,0.3170,0.5020,0.008964,0.014195
1,2021,0KPqjA456v,2,3395,2.8200,2.0227,0.079741,0.057196
2,2021,0KPqjA456v,3,3395,12.7276,8.2764,0.359897,0.234031
3,2021,0KPqjA456v,4,3395,1.4861,1.4773,0.042022,0.041773
4,2021,0KPqjA456v,5,3395,0.6393,0.8098,0.018077,0.022899
...,...,...,...,...,...,...,...,...
1645,2022,zeQZkL1MKw,26,2401,-0.0175,-0.2220,-0.000700,-0.008876
1646,2022,zeQZkL1MKw,27,2401,0.8140,0.7786,0.032546,0.031131
1647,2022,zeQZkL1MKw,28,2401,8.6677,10.9169,0.346564,0.436494
1648,2022,zeQZkL1MKw,29,2401,0.2932,2.4217,0.011723,0.096828


In [42]:
def find_transpose(season, zone, field):
    return grouped_gplus[(grouped_gplus.season_name == season) & (grouped_gplus.zone == (31 - zone))][field].tolist()[0]


grouped_gplus['net_p96'] = grouped_gplus['for_p96'] - grouped_gplus['against_p96']
grouped_gplus['net_total'] = grouped_gplus['for_total'] - grouped_gplus['against_total']
grouped_gplus['transposed_net_p96'] = grouped_gplus.apply(lambda x: x.for_p96 - find_transpose(x.season_name, x.zone, 'against_p96'), axis=1)
grouped_gplus['transposed_net_total'] = grouped_gplus.apply(lambda x: x.for_total - find_transpose(x.season_name, x.zone, 'against_total'), axis=1)
grouped_gplus

,season_name,team_id,zone,minutes,for_total,against_total,for_p96,against_p96,net_p96,net_total,transposed_net_p96,transposed_net_total
0,2021,0KPqjA456v,1,3395,0.3170,0.5020,0.008964,0.014195,-0.005231,-0.1850,0.014868,0.5258
1,2021,0KPqjA456v,2,3395,2.8200,2.0227,0.079741,0.057196,0.022545,0.7973,0.056166,1.9863
2,2021,0KPqjA456v,3,3395,12.7276,8.2764,0.359897,0.234031,0.125866,4.4512,-0.145530,-5.1466
3,2021,0KPqjA456v,4,3395,1.4861,1.4773,0.042022,0.041773,0.000249,0.0088,0.020537,0.7263
4,2021,0KPqjA456v,5,3395,0.6393,0.8098,0.018077,0.022899,-0.004821,-0.1705,0.013070,0.4622
...,...,...,...,...,...,...,...,...,...,...,...,...
1645,2022,zeQZkL1MKw,26,2401,-0.0175,-0.2220,-0.000700,-0.008876,0.008177,0.2045,-0.026213,-0.6524
1646,2022,zeQZkL1MKw,27,2401,0.8140,0.7786,0.032546,0.031131,0.001415,0.0354,-0.023342,-0.5768
1647,2022,zeQZkL1MKw,28,2401,8.6677,10.9169,0.346564,0.436494,-0.089931,-2.2492,0.067902,1.7331
1648,2022,zeQZkL1MKw,29,2401,0.2932,2.4217,0.011723,0.096828,-0.085105,-2.1285,-0.037543,-0.9328


In [43]:
base_range = np.linspace(0.01, 1.00, 100)
years = grouped_gplus["season_name"].unique().tolist()
zones = grouped_gplus["zone"].unique().tolist()
percentile_composite = pd.DataFrame()

def percentiles(zone, year):
    slice_gplus = grouped_gplus[(grouped_gplus.zone == zone) & (grouped_gplus.season_name == year)]
    if (len(slice_gplus) == 0):
        #print(f"no data for Combo of {position} / {action_type}")
        return pd.DataFrame()

    data = slice_gplus["for_total"]
    adj_data = slice_gplus["for_p96"]
    
    ag_data = slice_gplus["against_total"]
    ag_adj_data = slice_gplus["against_p96"]
    
    net_data = slice_gplus["net_total"]
    net_adj_data = slice_gplus["net_p96"]
        
    trans_net_data = slice_gplus["transposed_net_total"]
    trans_net_adj_data = slice_gplus["transposed_net_p96"]
    
    return pd.DataFrame({ 
        "season" : year,"zone" : zone, "pct" : base_range,
        "for_p96" : adj_data.quantile(base_range), "for_pSzn" : data.quantile(base_range),
        "against_p96" : ag_adj_data.quantile(base_range), "against_pSzn" : ag_data.quantile(base_range),
        "net_p96" : net_adj_data.quantile(base_range), "net_pSzn" : net_data.quantile(base_range),
        "trans_net_p96" : trans_net_adj_data.quantile(base_range), "trans_net_pSzn" : trans_net_data.quantile(base_range)
    })

for z in zones:
    for y in years:
        df = percentiles(z, y)
        percentile_composite = percentile_composite.append(df, ignore_index=True)
        
percentile_composite

,season,zone,pct,for_p96,for_pSzn,against_p96,against_pSzn,net_p96,net_pSzn,trans_net_p96,trans_net_pSzn
0,2021,1,0.01,0.007542,0.265584,0.011965,0.420448,-0.023620,-0.830602,0.013446,0.474384
1,2021,1,0.02,0.007829,0.275568,0.012030,0.421696,-0.020210,-0.710404,0.013733,0.484368
2,2021,1,0.03,0.008115,0.285552,0.012095,0.422944,-0.016801,-0.590206,0.014019,0.494352
3,2021,1,0.04,0.008382,0.294920,0.012168,0.424700,-0.013858,-0.486628,0.014286,0.503720
4,2021,1,0.05,0.008540,0.300900,0.012284,0.429250,-0.013484,-0.474460,0.014444,0.509700
...,...,...,...,...,...,...,...,...,...,...,...
5995,2022,30,0.96,0.020614,0.508716,0.020466,0.486672,0.039799,0.996668,0.000988,0.020316
5996,2022,30,0.97,0.028219,0.687607,0.023153,0.549176,0.043885,1.085947,0.008593,0.199207
5997,2022,30,0.98,0.038362,0.923938,0.026672,0.632984,0.048781,1.188898,0.018736,0.435538
5998,2022,30,0.99,0.048506,1.160269,0.030191,0.716792,0.053678,1.291849,0.028880,0.671869


In [44]:
percentile_composite.to_csv('./percentile-g+-zones.csv', index=False)